# Metrics

In order to compare the performance of the different models, we need to define a metric. This metric must take into account the following elements:
- Whether the simplified sentences are indeed at the expected level
- Whether the simplified sentences retain the meaning of the original sentence

To measure the first point, we simply use a model already trained in the `difficulty_estimation` section to estimate the difference in difficulty between the original sentences and the simplified sentences. A measure of **Accuracy** should enable us to obtain a score between 0 and 1 describing whether our first condition is met.

To measure the second point, we will use the **Similarity Consinus** between the embeddings of the original sentences and the simplified sentences. This measure will give us a score between 0 and 1 describing whether our second condition is met.

In order to remain consistent between these two measures and to use a different model to those we could evaluate (**Mistral-7B**, **Davinci**, **GPT3.5**), it seems appropriate to use a **BERT** model for both measures. La formule finale, inspirée du **F1-score** est la suivante :
$$
\begin{align*} \text{Metric} & = score = \frac{2 \times w_1 \times A \times (1 - w_1) \times S}{w_1 \times A + (1 - w_1) \times S } \\ \text{Where :} \\ \text{Metric} & = \text{The combined metric} \\ w_1 & = \text{Weighting coefficient for similarity} \\ w_2 & = (1-w_1) : \text{Weighting coefficient for accuracy}\\
S &= \text{Cosine similarity between input and output}\\
A &= \text{Accuracy between BERT predictions and expected labels}  \end{align*}
$$



In [1]:
# ---------------------------- PREPARING NOTEBOOK ---------------------------- #
# Autoreload
%load_ext autoreload
%autoreload 2

# Random seed
import numpy as np
np.random.seed(42)

# External modules
import os
from IPython.display import display

# Set global log level
import logging
logging.basicConfig(level=logging.INFO)
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

# Define PWD as the current git repository
import git
repo = git.Repo('.', search_parent_directories=True)
pwd = repo.working_dir
os.chdir(pwd)

# import

## Downloading test data

In order to evaluate the relevance of our metric, we will first evaluate it on the dataset used to fine-tune **Mistral-7B** in the [FineTuningMistral](b_FineTuningMistral.ipynb) notebook. This dataset is described in the [DatasetCreation](a_DatasetCreation.ipynb) notebook and was created using **GPT-4**.

In [2]:
# -------------------------- LOAD PREVIOUS NOTEBOOKS ------------------------- #
import json
import __main__
import black

paths = [
    os.path.join(pwd, "notebooks", "text_simplification", "a_DatasetCreation.ipynb"),
    os.path.join(pwd, "notebooks", "text_simplification", "c_MistralEvaluation.ipynb"),
]

# Read notebooks
code_dict = {}
for path in paths:
    code = ""
    with open(path, "r") as f:
        temp = json.load(f)

    cells = [
        cell
        for cell in temp["cells"]
        if cell["cell_type"] == "code"
        and len(cell["source"]) > 0
        and cell["source"][-1] == "# import"
    ]
    notebook_code = "\n".join(
        line
        for cell in cells
        for line in cell["source"]
        if line != "# import" and len(line) > 0 and line[0] != "%"
    )
    # Create something like a header
    code += f"# {'-'*76} #\n"
    code += f"# {os.path.basename(path).upper():^76} #\n"
    code += f"# {'-'*76} #\n"
    code += notebook_code

    # Add "Module Creation"
    notebook_name = (
        os.path.basename(path).replace("imported_", "").replace(".ipynb", "")
    )
    code += """
# --------------------------------- IMPORTER --------------------------------- #
import types


class MyNotebook:
    pass


NOTEBOOK_NAME = MyNotebook()
# Put every function defined in the notebook in the class
NOTEBOOK_NAME.__dict__.update(
    {
        name: obj
        for name, obj in locals().items()
        if isinstance(obj, (type, types.FunctionType))
        if not (name.startswith("_") or name == "MyNotebook")
    }
)
    """.replace(
        "NOTEBOOK_NAME", notebook_name
    )

    # Remove empty lines
    code = "\n".join([line for line in code.split("\n") if len(line) > 0])
    # Format code
    code = black.format_str(code, mode=black.FileMode())

    # Write scrach file
    path = os.path.join(
        pwd, "scratch", f"imported_{os.path.basename(path).replace('ipynb', 'py')}"
    )
    if not os.path.exists(os.path.dirname(path)):
        os.makedirs(os.path.dirname(path))
    with open(path, "w") as f:
        f.write(code)
    code_dict[path] = code


# Mainify code
for path, code in code_dict.items():
    compiled = compile(code, path, "exec")
    exec(compiled, __main__.__dict__)

# import

In [3]:
# ----------------------------- LOADING TEST DATA ---------------------------- #
metric_test_df = a_DatasetCreation.download_data()
metric_test_df.columns = ["Original", "Simplified"]
metric_test_df.head()

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

/home/lopilo24/UNIL/Lingorank/Lingorank_LLM/scratch/imported_a_DatasetCreation.py:71: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv(


,Original,Simplified
0,L'apprentissage des langues étrangères stimule...,On apprend mieux avec les langues étrangères.
1,Les écosystèmes marins sont régulièrement pert...,Les usines abîment souvent la vie sous la mer.
2,L'absorption de polluants atmosphériques par l...,Les forêts aident à garder l'air propre en abs...
3,"Confrontées à une mutation économique rapide, ...",Les entreprises doivent changer vite pour rest...
4,La philosophie existentialiste s'affirme par l...,L'existentialisme dit que la vie n'a pas de se...


## Difficulty assessment

We are going to use the **CamemBert** model already trained to evaluate the difficulty of sentences. We will define a function that takes a dataframe as input and returns a score between 0 and 1, indicating how well the difficulty of the original sentences are correctly related to the difficulty of the simplified sentences.

The dataframe to be evaluated must have the following format:
- `Original`: the original sentence
- `Simplified`': the simplified sentence

*Note that it is not necessary for the `Difficulty` column to be present in the dataframe, as we will calculate it in order to reduce the bias of our evaluation.*

In [4]:
# ------------------------- BERT PREDICTION FUNCTION ------------------------- #
import dill
import numpy as np
import pandas as pd
import torch
from huggingface_hub import snapshot_download
from transformers import CamembertForSequenceClassification
import os


def get_bert_difficulty_prediction(
    series: pd.Series, dataset: str, pwd: str = ".", probs: bool = False
):
    # Clone model checkpoint
    if not os.path.exists(os.path.join(pwd, dataset)):
        snapshot_download(
            repo_id=f"OloriBern/Lingorank_Bert_{dataset}",
            local_dir=os.path.join(pwd, dataset),
            revision="main",
            repo_type="model",
        )

    # Load tokenizer and label encoder
    with open(
        os.path.join(
            os.path.join(pwd, dataset),
            "train_camembert_tokenizer_label_encoder.pkl",
        ),
        "rb",
    ) as f:
        tokenizer, label_encoder = dill.load(f)

    # Charger le modèle; assurons-nous qu'il matche la classe de votre modèle
    model = CamembertForSequenceClassification.from_pretrained(
        os.path.join(pwd, dataset)
    )

    # Mettre le modèle en mode évaluation
    model.eval()

    # Préparer les données pour le modèle
    inputs = tokenizer(
        series.tolist(),
        padding=True,
        truncation=True,
        return_tensors="pt",
    )

    # Charger les tensors sur l'appareil adéquat (GPU si disponible)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Désactiver le calcul du gradient puisque nous sommes en inférence
    batch_size = 32
    all_predictions = []
    with torch.no_grad():
        # Traiter les inputs par batch pour éviter les MemoryError
        for i in range(0, len(inputs["input_ids"]), batch_size):
            batch_inputs = {
                key: value[i : i + batch_size].to(device)
                for key, value in inputs.items()
            }
            # Faire les prédictions
            outputs = model(**batch_inputs)

            # Appliquer une fonction softmax pour obtenir les probabilités
            predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

            # Convertir les prédictions en numpy array pour faciliter l'accès aux résultats et leur manipulation
            predictions = predictions.cpu().numpy()

            all_predictions.append(predictions)

    # Concatenate all predictions
    predictions = np.concatenate(all_predictions)

    # Get best predictions
    predictions_labels = np.argmax(predictions, axis=1)

    # Apply label encoder
    predictions_labels = label_encoder.inverse_transform(predictions_labels)

    if probs:
        return predictions_labels, predictions
    return predictions_labels


# import

In [5]:
# Create mini test set for BERT
mini_test = metric_test_df["Original"].sample(10)

# Predict
get_bert_difficulty_prediction(
    mini_test, "french_difficulty", os.path.join(pwd, "scratch"), probs=True
)

/home/lopilo24/UNIL/Lingorank/Lingorank_LLM/.venv/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.3.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


(array(['C2', 'B2', 'B2', 'B2', 'C1', 'B2', 'C1', 'C1', 'C2', 'C2'],
       dtype=object),
 array([[0.01383119, 0.00901323, 0.01455005, 0.16621284, 0.37233117,
         0.42406154],
        [0.01068887, 0.01366377, 0.04763468, 0.46639872, 0.33043745,
         0.13117649],
        [0.01425035, 0.02189387, 0.08409212, 0.5134722 , 0.27708477,
         0.08920669],
        [0.01271411, 0.01467501, 0.04439334, 0.45176357, 0.3529276 ,
         0.12352634],
        [0.01143033, 0.00865433, 0.01792894, 0.2653202 , 0.42384243,
         0.27282384],
        [0.01314038, 0.02212075, 0.09039756, 0.5195252 , 0.26459965,
         0.09021647],
        [0.01123757, 0.01046034, 0.02793905, 0.36819872, 0.3892517 ,
         0.19291267],
        [0.01180815, 0.01133882, 0.03001059, 0.3889259 , 0.39749056,
         0.16042599],
        [0.02519487, 0.01526103, 0.01788778, 0.10235687, 0.26662713,
         0.57267237],
        [0.02722683, 0.01667627, 0.0188247 , 0.09509498, 0.2472791 ,
         0.5948981 ]]

In [6]:
# ------------------------ GET SIMPLIFICATION ACCURACY ----------------------- #
import os
import pandas as pd
from typing import NamedTuple


def get_simplification_accuracy(
    df: pd.DataFrame, dataset: str, pwd: str = ".", m: bool = True
):
    # Get predictions
    predictions, probas = get_bert_difficulty_prediction(
        pd.concat([df["Original"], df["Simplified"]], axis=0), dataset, pwd, probs=True
    )

    comparison = {
        "Original": probas[: len(probas) // 2, 1:],  # Remove the first column
        "Simplified": np.cumsum(
            probas[len(probas) // 2 :, :-1], axis=1
        ),  # Remove the last column
    }

    results = (comparison["Original"] * comparison["Simplified"]).sum(axis=1)
    if m:
        accuracy = results.mean()
    else:
        accuracy = results

    # Return accuracy & predictions
    return NamedTuple(
        "SimplificationAccuracy", [("accuracy", float), ("predictions", pd.DataFrame)]
    )(accuracy, comparison)

In [7]:
get_simplification_accuracy(
    metric_test_df, "french_difficulty", os.path.join(pwd, "scratch")
).accuracy

/home/lopilo24/UNIL/Lingorank/Lingorank_LLM/.venv/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.3.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


0.6349311

## Evaluation of the conservation of meaning

We now need to check whether the simplified sentences retain the meaning of the original sentences. To do this, we're going to apply the same **CamemBERT** model, but this time without any fine-tuning to compare the embeddings of each sentence.

In [8]:
# ----------------------- GET SIMPLIFICATION SIMILARITY ---------------------- #
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from typing import NamedTuple


def calculate_bleu_score(original, simplified):
    reference = original.split()
    candidate = simplified.split()
    score = sentence_bleu([reference], candidate)
    return score


def calculate_rouge_score(original, simplified):
    scorer = rouge_scorer.RougeScorer(["rouge1", "rougeL"], use_stemmer=True)
    scores = scorer.score(original, simplified)
    return scores


def get_simplification_similarity(
    df: pd.DataFrame, bleu: bool = False, rouge: bool = False
):
    if bleu and rouge:
        raise ValueError("Cannot use both BLEU and ROUGE at the same time")
    elif bleu:
        similarity = df.dropna().apply(
            lambda row: calculate_bleu_score(row["Original"], row["Simplified"]), axis=1
        )
    elif rouge:
        similarity = (
            df.dropna()
            .apply(
                lambda row: sum(
                    [
                        x[2]
                        for x in calculate_rouge_score(
                            row["Original"], row["Simplified"]
                        ).values()
                    ]
                )
                / 2,
                axis=1,
            )
            .iloc[0]
        )
    else:
        # Load model
        model = SentenceTransformer("camembert-base")

        # Encode sentences
        original = model.encode(df["Original"].tolist())
        simplified = model.encode(df["Simplified"].tolist())

        # Compute cosine similarity
        similarity = cosine_similarity(original, simplified)

        # Keep only diagonal
        similarity = np.diag(similarity)

    # Mean similarity
    similarity_mean = similarity.mean()

    # Return similarity & similarity mean
    return NamedTuple(
        "SimplificationSimilarity",
        [("similarity", np.ndarray), ("similarity_mean", float)],
    )(similarity, similarity_mean)


# import

In [9]:
print(get_simplification_similarity(metric_test_df).similarity_mean)
print(get_simplification_similarity(metric_test_df, bleu=True).similarity_mean)
print(get_simplification_similarity(metric_test_df, rouge=True).similarity_mean)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: camembert-base
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

/home/lopilo24/UNIL/Lingorank/Lingorank_LLM/.venv/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/lopilo24/UNIL/Lingorank/Lingorank_LLM/.venv/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/lopilo24/UNIL/Lingorank/Lingorank_LLM/.venv/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram o

0.8856876
0.01240844735738772
0.38888888888888884


## Definition of the metric

We have functions that can give us similarity and difficulty scores. So we're going to define a function that takes a dataframe as input and returns the metric score.

In [10]:
# ------------------------------- DEFINE METRIC ------------------------------ #
def simplification_metric(
    df: pd.DataFrame,
    dataset: str,
    pwd: str = ".",
    w1: float = 0.5,
    bleu: bool = False,
    rouge: bool = False,
):
    accuracy = get_simplification_accuracy(df, dataset, pwd).accuracy
    similarity = get_simplification_similarity(
        df, bleu=bleu, rouge=rouge
    ).similarity_mean

    score = (
        4
        * (w1 * accuracy * (1 - w1) * similarity)
        / (w1 * accuracy + (1 - w1) * similarity)
    )

    return NamedTuple(
        "SimplificationMetric",
        [("accuracy", float), ("similarity", float), ("score", float)],
    )(accuracy, similarity, score)

In [ ]:
def get_simplification_accuracy(df: pd.DataFrame) ->
pd.Series:↩→
# Get predictions
logits = get_bert_difficulty_prediction(
pd.concat([df["Original"], df["Simplified"]],
axis=0)↩→
)
comparison = {
"Original": logits[: len(logits) // 2, 1:], #
Remove the first column↩→
"Simplified": np.cumsum(logits[len(logits) // 2 :,
:-1], axis=1), # Remove the last column↩→
}
# The probability that the text is at each level is
multiplied by the probability that its simplified
version is at least one level lower.
↩→
↩→
results = (comparison["Original"] *
comparison["Simplified"]).sum(axis=1)↩→
return results

### Evaluation of the metric

To evaluate the relevance of our metric, we will apply it to four datasets:
- The dataset used for fine-tuning **Mistral-7B** (*Supposed to have a higher score*)
- A test dataset composed of 60 sentences simplified by **Mistral-7B** (*Supposed to have a lower score*)
- A dataset of 60 unsimplified sentences.
- A dataset consisting of 60 sentences associated with another sentence at a lower level but with no semantic link.

In [11]:
# ------------------------- CREATING EVALUATION SETS ------------------------- #
# GPT-4
test_df_1 = metric_test_df

# Mistral-7B
test_df_2 = pd.read_csv(
    os.path.join(
        pwd,
        "results",
        "text_simplification",
        "MistralEvaluation",
        "predictions_fine_tuned_formatted.csv",
    )
)

# Perfect Meaning
test_df_original = c_MistralEvaluation.get_balanced_dataframe(
    c_MistralEvaluation.download_difficulty_estimation("."), 100
)
test_df_3 = pd.concat(
    [test_df_original["Sentence"], test_df_original["Sentence"]], axis=1
)
test_df_3.columns = ["Original", "Simplified"]

# Perfect Simplification
test_df_4 = (
    pd.concat(
        [test_df_original["Sentence"], test_df_original["Sentence"].shift(10)],
        axis=1,
    )
    .dropna()
    .reset_index(drop=True)
)
test_df_4.columns = ["Original", "Simplified"]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

In [12]:
# Mute UserWarning
import warnings

warnings.filterwarnings("ignore")

print(get_simplification_similarity(test_df_1, bleu=True).similarity_mean)
print(get_simplification_similarity(test_df_2, bleu=True).similarity_mean)
print(get_simplification_similarity(test_df_3, bleu=True).similarity_mean)
print(get_simplification_similarity(test_df_4, bleu=True).similarity_mean)

0.01240844735738772
0.0748661577512338
0.98
8.228289207862188e-05


In [13]:
# ----------------------------- METRIC ASSESSMENT ---------------------------- #
# Loading test data
test_df_1 = metric_test_df
test_df_2 = pd.read_csv(
    os.path.join(
        pwd,
        "results",
        "text_simplification",
        "MistralEvaluation",
        "predictions_fine_tuned_formatted.csv",
    )
)

# Assessing metric
## Bert
gpt4_metric_best = simplification_metric(
    test_df_1, "french_difficulty", os.path.join(pwd, "scratch")
)
mistral_metric_best = simplification_metric(
    test_df_2.dropna(axis=0), "french_difficulty", os.path.join(pwd, "scratch")
)
perfect_meaning_metric_best = simplification_metric(
    test_df_3, "french_difficulty", os.path.join(pwd, "scratch")
)
perfect_simplification_metric_best = simplification_metric(
    test_df_4, "french_difficulty", os.path.join(pwd, "scratch")
)
## Blue
gpt4_metric_best_bleu = simplification_metric(
    test_df_1, "french_difficulty", os.path.join(pwd, "scratch"), bleu=True
)
mistral_metric_best_bleu = simplification_metric(
    test_df_2.dropna(axis=0),
    "french_difficulty",
    os.path.join(pwd, "scratch"),
    bleu=True,
)
perfect_meaning_metric_best_bleu = simplification_metric(
    test_df_3, "french_difficulty", os.path.join(pwd, "scratch"), bleu=True
)
perfect_simplification_metric_best_bleu = simplification_metric(
    test_df_4, "french_difficulty", os.path.join(pwd, "scratch"), bleu=True
)
## Rouge
gpt4_metric_best_rouge = simplification_metric(
    test_df_1, "french_difficulty", os.path.join(pwd, "scratch"), rouge=True
)
mistral_metric_best_rouge = simplification_metric(
    test_df_2.dropna(axis=0),
    "french_difficulty",
    os.path.join(pwd, "scratch"),
    rouge=True,
)
perfect_meaning_metric_best_rouge = simplification_metric(
    test_df_3, "french_difficulty", os.path.join(pwd, "scratch"), rouge=True
)
perfect_simplification_metric_best_rouge = simplification_metric(
    test_df_4, "french_difficulty", os.path.join(pwd, "scratch"), rouge=True
)

# Displaying results
print(f"Test 1 (GPT-4): {gpt4_metric_best}, {gpt4_metric_best_bleu}")
print(f"Test 2 (Mistral-7B): {mistral_metric_best}, {mistral_metric_best_bleu}")
print(
    f"Test 3 (Perfect Meaning): {perfect_meaning_metric_best}, {perfect_meaning_metric_best_bleu}"
)
print(
    f"Test 4 (Perfect Simplification): {perfect_simplification_metric_best}, {perfect_simplification_metric_best_bleu}"
)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: camembert-base
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: camembert-base
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda


Batches:   0%|          | 0/32 [00:00<?, ?it/s]

...

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: camembert-base
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda


Batches:   0%|          | 0/31 [00:00<?, ?it/s]

Batches:   0%|          | 0/31 [00:00<?, ?it/s]

INFO:absl:Using default tokenizer.
INFO:absl:Using default tokenizer.
INFO:absl:Using default tokenizer.
INFO:absl:Using default tokenizer.
INFO:absl:Using default tokenizer.
INFO:absl:Using default tokenizer.
INFO:absl:Using default tokenizer.
INFO:absl:Using default tokenizer.
INFO:absl:Using default tokenizer.
INFO:absl:Using default tokenizer.
INFO:absl:Using default tokenizer.
INFO:absl:Using default tokenizer.
INFO:absl:Using default tokenizer.
INFO:absl:Using default tokenizer.
INFO:absl:Using default tokenizer.
INFO:absl:Using default tokenizer.
INFO:absl:Using default tokenizer.
INFO:absl:Using default tokenizer.
INFO:absl:Using default tokenizer.
INFO:absl:Using default tokenizer.
INFO:absl:Using default tokenizer.
INFO:absl:Using default tokenizer.
INFO:absl:Using default tokenizer.
INFO:absl:Using default tokenizer.
INFO:absl:Using default tokenizer.
INFO:absl:Using default tokenizer.
INFO:absl:Using default tokenizer.
INFO:absl:Using default tokenizer.
INFO:absl:Using defa

Test 1 (GPT-4): SimplificationMetric(accuracy=0.6349311, similarity=0.8856876, score=0.7396339304885228), SimplificationMetric(accuracy=0.6349311, similarity=0.01240844735738772, score=0.024341195155172808)
Test 2 (Mistral-7B): SimplificationMetric(accuracy=0.59397876, similarity=0.9138029, score=0.7199709803813235), SimplificationMetric(accuracy=0.59397876, similarity=0.0748661577512338, score=0.1329722522780479)
Test 3 (Perfect Meaning): SimplificationMetric(accuracy=0.3154044, similarity=1.0, score=0.4795550156989565), SimplificationMetric(accuracy=0.3154044, similarity=0.98, score=0.47721977958781275)
Test 4 (Perfect Simplification): SimplificationMetric(accuracy=0.36129043, similarity=0.80995727, score=0.4996890190386234), SimplificationMetric(accuracy=0.36129043, similarity=8.228289207862188e-05, score=0.00016452831329029783)


### Find ideal parameters

To find the best parameters for our metric, we simply want to ensure that the scores obtained for datasets `Test 3` & `Test 4` are equal. After a few simplifications, we want $$w_1=\dfrac{y_2-y_1}{x_1-x_2+y_2-y_1}$$. Now all we have to do is the maths!

In [16]:
# ----------------------------- METRIC ASSESSMENT ---------------------------- #
# Find best w1
## Bert
x_1 = perfect_meaning_metric_best.accuracy
x_2 = perfect_simplification_metric_best.accuracy
y_1 = perfect_meaning_metric_best.similarity
y_2 = perfect_simplification_metric_best.similarity
w_1_bert = (y_2 - y_1) / (x_1 - x_2 + y_2 - y_1)
## BLEU
x_1 = perfect_meaning_metric_best_bleu.accuracy
x_2 = perfect_simplification_metric_best_bleu.accuracy
y_1 = perfect_meaning_metric_best_bleu.similarity
y_2 = perfect_simplification_metric_best_bleu.similarity
w_1_bleu = (y_2 - y_1) / (x_1 - x_2 + y_2 - y_1)
# ROUGE
x_1 = perfect_meaning_metric_best_rouge.accuracy
x_2 = perfect_simplification_metric_best_rouge.accuracy
y_1 = perfect_meaning_metric_best_rouge.similarity
y_2 = perfect_simplification_metric_best_rouge.similarity
w_1_rouge = (y_2 - y_1) / (x_1 - x_2 + y_2 - y_1)
logging.info(f"Best w1 (BERT): {w_1_bert}")
logging.info(f"Best w1 (BLUE): {w_1_bleu}")
logging.info(f"Best w1 (ROUGE): {w_1_rouge}")

INFO:root:Best w1 (BERT): 0.8055089116096497
INFO:root:Best w1 (BLUE): 0.955268208536145
INFO:root:Best w1 (ROUGE): 0.9561271132889757


## Calculation of all metrics

Now that we have a good evaluation of our metric, we can apply it to all the models we trained in the previous notebooks. Namely :
- **Mistral-7B**
- **Davinci Zero-Shot**
- **GPT-3.5 Zero-Shot**
- **Davinci Fine-Tuned**
- **GPT-3.5 Fine-Tuned**

For an effective comparison, we will also evaluate the models on the same datasets as for the metric evaluation (**Perfect Meaning** & **Perfect Evaluation**).

In [18]:
# ----------------------------- COMPUTING METRICS ---------------------------- #
from IPython.display import Markdown

# Set w1 to best value because of incorrect Perfect Simplification
w_1_bert = 0.5
w_1_bleu = 0.5
w_1_rouge = 0.5

# Perfect Simplification
display(Markdown("### Perfect Simplification"))
display(test_df_4.head())
perfect_simplification_metric_bert = simplification_metric(
    test_df_4, "french_difficulty", os.path.join(pwd, "scratch"), w1=w_1_bert
)
perfect_simplification_metric_bleu = simplification_metric(
    test_df_4, "french_difficulty", os.path.join(pwd, "scratch"), w1=w_1_bleu, bleu=True
)
perfect_simplification_metric_rouge = simplification_metric(
    test_df_4,
    "french_difficulty",
    os.path.join(pwd, "scratch"),
    w1=w_1_rouge,
    rouge=True,
)
display(
    Markdown(
        f"**Accuracy (BERT)**: *{perfect_simplification_metric_bert.accuracy:.2%}*"
    )
)
display(
    Markdown(
        f"**Similarity (BERT)**: *{perfect_simplification_metric_bert.similarity:.2%}*"
    )
)
display(Markdown(f"**Score (BERT)**: *{perfect_simplification_metric_bert.score:.2%}*"))
display(
    Markdown(
        f"**Accuracy (BLEU)**: *{perfect_simplification_metric_bleu.accuracy:.2%}*"
    )
)
display(
    Markdown(
        f"**Similarity (BLEU)**: *{perfect_simplification_metric_bleu.similarity:.2%}*"
    )
)
display(Markdown(f"**Score (BLEU)**: *{perfect_simplification_metric_bleu.score:.2%}*"))
display(
    Markdown(
        f"**Accuracy (ROUGE)**: *{perfect_simplification_metric_rouge.accuracy:.2%}*"
    )
)
display(
    Markdown(
        f"**Similarity (ROUGE)**: *{perfect_simplification_metric_rouge.similarity:.2%}*"
    )
)
display(
    Markdown(f"**Score (ROUGE)**: *{perfect_simplification_metric_rouge.score:.2%}*")
)

# Perfect Meaning
display(Markdown("### Perfect Meaning"))
display(test_df_3.head())
perfect_meaning_metric_bert = simplification_metric(
    test_df_3, "french_difficulty", os.path.join(pwd, "scratch"), w1=w_1_bert
)
perfect_meaning_metric_bleu = simplification_metric(
    test_df_3, "french_difficulty", os.path.join(pwd, "scratch"), w1=w_1_bleu, bleu=True
)
perfect_meaning_metric_rouge = simplification_metric(
    test_df_3,
    "french_difficulty",
    os.path.join(pwd, "scratch"),
    w1=w_1_rouge,
    rouge=True,
)
display(Markdown(f"**Accuracy (BERT)**: *{perfect_meaning_metric_bert.accuracy:.2%}*"))
display(
    Markdown(f"**Similarity (BERT)**: *{perfect_meaning_metric_bert.similarity:.2%}*")
)
display(Markdown(f"**Score (BERT)**: *{perfect_meaning_metric_bert.score:.2%}*"))
display(Markdown(f"**Accuracy (BLEU)**: *{perfect_meaning_metric_bleu.accuracy:.2%}*"))
display(
    Markdown(f"**Similarity (BLEU)**: *{perfect_meaning_metric_bleu.similarity:.2%}*")
)
display(Markdown(f"**Score (BLEU)**: *{perfect_meaning_metric_bleu.score:.2%}*"))
display(
    Markdown(f"**Accuracy (ROUGE)**: *{perfect_meaning_metric_rouge.accuracy:.2%}*")
)
display(
    Markdown(f"**Similarity (ROUGE)**: *{perfect_meaning_metric_rouge.similarity:.2%}*")
)
display(Markdown(f"**Score (ROUGE)**: *{perfect_meaning_metric_rouge.score:.2%}*"))

# GPT-4
display(Markdown("### GPT-4"))
display(metric_test_df.head())
gpt4_metric_bert = simplification_metric(
    metric_test_df, "french_difficulty", os.path.join(pwd, "scratch"), w1=w_1_bert
)
gpt4_metric_bleu = simplification_metric(
    metric_test_df,
    "french_difficulty",
    os.path.join(pwd, "scratch"),
    w1=w_1_bleu,
    bleu=True,
)
gpt4_metric_rouge = simplification_metric(
    metric_test_df,
    "french_difficulty",
    os.path.join(pwd, "scratch"),
    w1=w_1_rouge,
    rouge=True,
)
display(Markdown(f"**Accuracy (BERT)**: *{gpt4_metric_bert.accuracy:.2%}*"))
display(Markdown(f"**Similarity (BERT)**: *{gpt4_metric_bert.similarity:.2%}*"))
display(Markdown(f"**Score (BERT)**: *{gpt4_metric_bert.score:.2%}*"))
display(Markdown(f"**Accuracy (BLEU)**: *{gpt4_metric_bleu.accuracy:.2%}*"))
display(Markdown(f"**Similarity (BLEU)**: *{gpt4_metric_bleu.similarity:.2%}*"))
display(Markdown(f"**Score (BLEU)**: *{gpt4_metric_bleu.score:.2%}*"))
display(Markdown(f"**Accuracy (ROUGE)**: *{gpt4_metric_rouge.accuracy:.2%}*"))
display(Markdown(f"**Similarity (ROUGE)**: *{gpt4_metric_rouge.similarity:.2%}*"))
display(Markdown(f"**Score (ROUGE)**: *{gpt4_metric_rouge.score:.2%}*"))

# Mistral-7B Zero-shot
path = os.path.join(pwd, "results", "text_simplification")
mistral_zero_shot_df = pd.read_csv(
    os.path.join(path, "MistralEvaluation", "predictions_zero_shot_formatted.csv")
)
display(Markdown("### Mistral-7B Zero-shot"))
display(mistral_zero_shot_df.head())
mistral_zero_shot_metric_bert = simplification_metric(
    mistral_zero_shot_df.dropna(),
    "french_difficulty",
    os.path.join(pwd, "scratch"),
    w1=w_1_bert,
)
mistral_zero_shot_metric_bleu = simplification_metric(
    mistral_zero_shot_df.dropna(),
    "french_difficulty",
    os.path.join(pwd, "scratch"),
    w1=w_1_bleu,
    bleu=True,
)
mistral_zero_shot_metric_rouge = simplification_metric(
    mistral_zero_shot_df.dropna(),
    "french_difficulty",
    os.path.join(pwd, "scratch"),
    w1=w_1_rouge,
    rouge=True,
)
display(
    Markdown(f"**Accuracy (BERT)**: *{mistral_zero_shot_metric_bert.accuracy:.2%}*")
)
display(
    Markdown(f"**Similarity (BERT)**: *{mistral_zero_shot_metric_bert.similarity:.2%}*")
)
display(Markdown(f"**Score (BERT)**: *{mistral_zero_shot_metric_bert.score:.2%}*"))
display(
    Markdown(f"**Accuracy (BLEU)**: *{mistral_zero_shot_metric_bleu.accuracy:.2%}*")
)
display(
    Markdown(f"**Similarity (BLEU)**: *{mistral_zero_shot_metric_bleu.similarity:.2%}*")
)
display(Markdown(f"**Score (BLEU)**: *{mistral_zero_shot_metric_bleu.score:.2%}*"))
display(
    Markdown(f"**Accuracy (ROUGE)**: *{mistral_zero_shot_metric_rouge.accuracy:.2%}*")
)
display(
    Markdown(
        f"**Similarity (ROUGE)**: *{mistral_zero_shot_metric_rouge.similarity:.2%}*"
    )
)
display(Markdown(f"**Score (ROUGE)**: *{mistral_zero_shot_metric_rouge.score:.2%}*"))

# Mistral-7B Fine-tuned
path = os.path.join(pwd, "results", "text_simplification")
mistral_fine_tuned_df = pd.read_csv(
    os.path.join(path, "MistralEvaluation", "predictions_fine_tuned_formatted.csv")
)
display(Markdown("### Mistral-7B Fine-tuned"))
display(mistral_fine_tuned_df.head())
mistral_fine_tuned_metric_bert = simplification_metric(
    mistral_fine_tuned_df.dropna(),
    "french_difficulty",
    os.path.join(pwd, "scratch"),
    w1=w_1_bert,
)
mistral_fine_tuned_metric_bleu = simplification_metric(
    mistral_fine_tuned_df.dropna(),
    "french_difficulty",
    os.path.join(pwd, "scratch"),
    w1=w_1_bleu,
    bleu=True,
)
mistral_fine_tuned_metric_rouge = simplification_metric(
    mistral_fine_tuned_df.dropna(),
    "french_difficulty",
    os.path.join(pwd, "scratch"),
    w1=w_1_rouge,
    rouge=True,
)
display(
    Markdown(f"**Accuracy (BERT)**: *{mistral_fine_tuned_metric_bert.accuracy:.2%}*")
)
display(
    Markdown(
        f"**Similarity (BERT)**: *{mistral_fine_tuned_metric_bert.similarity:.2%}*"
    )
)
display(Markdown(f"**Score (BERT)**: *{mistral_fine_tuned_metric_bert.score:.2%}*"))
display(
    Markdown(f"**Accuracy (BLEU)**: *{mistral_fine_tuned_metric_bleu.accuracy:.2%}*")
)
display(
    Markdown(
        f"**Similarity (BLEU)**: *{mistral_fine_tuned_metric_bleu.similarity:.2%}*"
    )
)
display(Markdown(f"**Score (BLEU)**: *{mistral_fine_tuned_metric_bleu.score:.2%}*"))
display(
    Markdown(f"**Accuracy (ROUGE)**: *{mistral_fine_tuned_metric_rouge.accuracy:.2%}*")
)
display(
    Markdown(
        f"**Similarity (ROUGE)**: *{mistral_fine_tuned_metric_rouge.similarity:.2%}*"
    )
)
display(Markdown(f"**Score (ROUGE)**: *{mistral_fine_tuned_metric_rouge.score:.2%}*"))

# GPT4 Zero-Shot
gpt4_zero_shot_df = pd.read_csv(
    os.path.join(
        path,
        "OpenAIEvaluation",
        "gpt-4-turbo-2024-04-09-zero-shot_formatted_predictions.csv",
    )
)
display(Markdown("### GPT4 Zero-Shot"))
display(gpt4_zero_shot_df.head())
gpt4_zero_shot_metric_bert = simplification_metric(
    gpt4_zero_shot_df.dropna(),
    "french_difficulty",
    os.path.join(pwd, "scratch"),
    w1=w_1_bert,
)
gpt4_zero_shot_metric_bleu = simplification_metric(
    gpt4_zero_shot_df.dropna(),
    "french_difficulty",
    os.path.join(pwd, "scratch"),
    w1=w_1_bleu,
    bleu=True,
)
gpt4_zero_shot_metric_rouge = simplification_metric(
    gpt4_zero_shot_df.dropna(),
    "french_difficulty",
    os.path.join(pwd, "scratch"),
    w1=w_1_rouge,
    rouge=True,
)
display(Markdown(f"**Accuracy (BERT)**: *{gpt4_zero_shot_metric_bert.accuracy:.2%}*"))
display(
    Markdown(f"**Similarity (BERT)**: *{gpt4_zero_shot_metric_bert.similarity:.2%}*")
)
display(Markdown(f"**Score (BERT)**: *{gpt4_zero_shot_metric_bert.score:.2%}*"))
display(Markdown(f"**Accuracy (BLEU)**: *{gpt4_zero_shot_metric_bleu.accuracy:.2%}*"))
display(
    Markdown(f"**Similarity (BLEU)**: *{gpt4_zero_shot_metric_bleu.similarity:.2%}*")
)
display(Markdown(f"**Score (BLEU)**: *{gpt4_zero_shot_metric_bleu.score:.2%}*"))
display(Markdown(f"**Accuracy (ROUGE)**: *{gpt4_zero_shot_metric_rouge.accuracy:.2%}*"))
display(
    Markdown(f"**Similarity (ROUGE)**: *{gpt4_zero_shot_metric_rouge.similarity:.2%}*")
)
display(Markdown(f"**Score (ROUGE)**: *{gpt4_zero_shot_metric_rouge.score:.2%}*"))

# GPT4-o
gpt4_o_df = pd.read_csv(
    os.path.join(
        path,
        "OpenAIEvaluation",
        "gpt-4o-2024-05-13-zero-shot_formatted_predictions.csv",
    )
)
display(Markdown("### GPT4-o Zero-Shot"))
display(gpt4_o_df.head())
gpt4_o_metric_bert = simplification_metric(
    gpt4_o_df.dropna(),
    "french_difficulty",
    os.path.join(pwd, "scratch"),
    w1=w_1_bert,
)
gpt4_o_metric_bleu = simplification_metric(
    gpt4_o_df.dropna(),
    "french_difficulty",
    os.path.join(pwd, "scratch"),
    w1=w_1_bleu,
    bleu=True,
)
gpt4_o_metric_rouge = simplification_metric(
    gpt4_o_df.dropna(),
    "french_difficulty",
    os.path.join(pwd, "scratch"),
    w1=w_1_rouge,
    rouge=True,
)
display(Markdown(f"**Accuracy (BERT)**: *{gpt4_o_metric_bert.accuracy:.2%}*"))
display(Markdown(f"**Similarity (BERT)**: *{gpt4_o_metric_bert.similarity:.2%}*"))
display(Markdown(f"**Score (BERT)**: *{gpt4_o_metric_bert.score:.2%}*"))
display(Markdown(f"**Accuracy (BLEU)**: *{gpt4_o_metric_bleu.accuracy:.2%}*"))
display(Markdown(f"**Similarity (BLEU)**: *{gpt4_o_metric_bleu.similarity:.2%}*"))
display(Markdown(f"**Score (BLEU)**: *{gpt4_o_metric_bleu.score:.2%}*"))
display(Markdown(f"**Accuracy (ROUGE)**: *{gpt4_o_metric_rouge.accuracy:.2%}*"))
display(Markdown(f"**Similarity (ROUGE)**: *{gpt4_o_metric_rouge.similarity:.2%}*"))
display(Markdown(f"**Score (ROUGE)**: *{gpt4_o_metric_rouge.score:.2%}*"))

# GPT Zero-Shot
gpt_df = pd.read_csv(
    os.path.join(
        path, "OpenAIEvaluation", "gpt-3.5-turbo-1106_zeroshot_predictions.csv"
    )
)
display(Markdown("### GPT Zero-Shot"))
display(gpt_df.head())
gpt_zero_shot_metric_bert = simplification_metric(
    gpt_df.dropna(),
    "french_difficulty",
    os.path.join(pwd, "scratch"),
    w1=w_1_bert,
)
gpt_zero_shot_metric_bleu = simplification_metric(
    gpt_df.dropna(),
    "french_difficulty",
    os.path.join(pwd, "scratch"),
    w1=w_1_bleu,
    bleu=True,
)
gpu_zero_shot_metric_rouge = simplification_metric(
    gpt_df.dropna(),
    "french_difficulty",
    os.path.join(pwd, "scratch"),
    w1=w_1_rouge,
    rouge=True,
)
display(Markdown(f"**Accuracy (BERT)**: *{gpt_zero_shot_metric_bert.accuracy:.2%}*"))
display(
    Markdown(f"**Similarity (BERT)**: *{gpt_zero_shot_metric_bert.similarity:.2%}*")
)
display(Markdown(f"**Score (BERT)**: *{gpt_zero_shot_metric_bert.score:.2%}*"))
display(Markdown(f"**Accuracy (BLEU)**: *{gpt_zero_shot_metric_bleu.accuracy:.2%}*"))
display(
    Markdown(f"**Similarity (BLEU)**: *{gpt_zero_shot_metric_bleu.similarity:.2%}*")
)
display(Markdown(f"**Score (BLEU)**: *{gpt_zero_shot_metric_bleu.score:.2%}*"))
display(Markdown(f"**Accuracy (ROUGE)**: *{gpu_zero_shot_metric_rouge.accuracy:.2%}*"))
display(
    Markdown(f"**Similarity (ROUGE)**: *{gpu_zero_shot_metric_rouge.similarity:.2%}*")
)
display(Markdown(f"**Score (ROUGE)**: *{gpu_zero_shot_metric_rouge.score:.2%}*"))


# Davinci Fine-Tuned
davinci_fn_df = pd.read_csv(
    os.path.join(
        path, "OpenAIEvaluation", "davinci-002-finetuned_formatted_predictions.csv"
    )
)
display(Markdown("### Davinci Fine-Tuned"))
display(davinci_fn_df.head())
davinci_finetuned_metric_bert = simplification_metric(
    davinci_fn_df.dropna(),
    "french_difficulty",
    os.path.join(pwd, "scratch"),
    w1=w_1_bert,
)
davinci_finetuned_metric_bleu = simplification_metric(
    davinci_fn_df.dropna(),
    "french_difficulty",
    os.path.join(pwd, "scratch"),
    w1=w_1_bleu,
    bleu=True,
)
daivnci_finetuned_metric_rouge = simplification_metric(
    davinci_fn_df.dropna(),
    "french_difficulty",
    os.path.join(pwd, "scratch"),
    w1=w_1_rouge,
    rouge=True,
)
display(
    Markdown(f"**Accuracy (BERT)**: *{davinci_finetuned_metric_bert.accuracy:.2%}*")
)
display(
    Markdown(f"**Similarity (BERT)**: *{davinci_finetuned_metric_bert.similarity:.2%}*")
)
display(Markdown(f"**Score (BERT)**: *{davinci_finetuned_metric_bert.score:.2%}*"))
display(
    Markdown(f"**Accuracy (BLEU)**: *{davinci_finetuned_metric_bleu.accuracy:.2%}*")
)
display(
    Markdown(f"**Similarity (BLEU)**: *{davinci_finetuned_metric_bleu.similarity:.2%}*")
)
display(Markdown(f"**Score (BLEU)**: *{davinci_finetuned_metric_bleu.score:.2%}*"))
display(
    Markdown(f"**Accuracy (ROUGE)**: *{daivnci_finetuned_metric_rouge.accuracy:.2%}*")
)
display(
    Markdown(
        f"**Similarity (ROUGE)**: *{daivnci_finetuned_metric_rouge.similarity:.2%}*"
    )
)
display(Markdown(f"**Score (ROUGE)**: *{daivnci_finetuned_metric_rouge.score:.2%}*"))

# GPT Fine-Tuned
gpt_fn_df = pd.read_csv(
    os.path.join(
        path,
        "OpenAIEvaluation",
        "gpt-3.5-turbo-1106-finetuned_formatted_predictions.csv",
    )
)
display(Markdown("### GPT Fine-Tuned"))
display(gpt_fn_df.head())
gpt_finetuned_metric_bert = simplification_metric(
    gpt_fn_df.dropna(),
    "french_difficulty",
    os.path.join(pwd, "scratch"),
    w1=w_1_bert,
)
gpt_finetuned_metric_bleu = simplification_metric(
    gpt_fn_df.dropna(),
    "french_difficulty",
    os.path.join(pwd, "scratch"),
    w1=w_1_bleu,
    bleu=True,
)
gpt_finetuned_metric_rouge = simplification_metric(
    gpt_fn_df.dropna(),
    "french_difficulty",
    os.path.join(pwd, "scratch"),
    w1=w_1_rouge,
    rouge=True,
)
display(Markdown(f"**Accuracy (BERT)**: *{gpt_finetuned_metric_bert.accuracy:.2%}*"))
display(
    Markdown(f"**Similarity (BERT)**: *{gpt_finetuned_metric_bert.similarity:.2%}*")
)
display(Markdown(f"**Score (BERT)**: *{gpt_finetuned_metric_bert.score:.2%}*"))
display(Markdown(f"**Accuracy (BLEU)**: *{gpt_finetuned_metric_bleu.accuracy:.2%}*"))
display(
    Markdown(f"**Similarity (BLEU)**: *{gpt_finetuned_metric_bleu.similarity:.2%}*")
)
display(Markdown(f"**Score (BLEU)**: *{gpt_finetuned_metric_bleu.score:.2%}*"))
display(Markdown(f"**Accuracy (ROUGE)**: *{gpt_finetuned_metric_rouge.accuracy:.2%}*"))
display(
    Markdown(f"**Similarity (ROUGE)**: *{gpt_finetuned_metric_rouge.similarity:.2%}*")
)
display(Markdown(f"**Score (ROUGE)**: *{gpt_finetuned_metric_rouge.score:.2%}*"))

### Perfect Simplification

,Original,Simplified
0,Certains champignons rouges à pois jaunes deva...,Le couple a bien fait d'évacuer
1,Il savait que si un nouveau conflit avec les E...,"— Voyez-vous, dit-il, l’avantage des tempêtes,..."
2,"Contre cette mauvaise pente, il ne connaissait...",Le feuillage des arbres faisait comme une mer ...
3,Cent fois le bois se fendit sous l’action soit...,Le sabre d’abordage et la hache étaient égalem...
4,Mais il aurait fallu des milliers de cages de ...,"Il travaillait lentement, soigneusement, assem..."


INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: camembert-base
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda


Batches:   0%|          | 0/31 [00:00<?, ?it/s]

Batches:   0%|          | 0/31 [00:00<?, ?it/s]

...

**Similarity (BLEU)**: *4.13%*

**Score (BLEU)**: *7.70%*

**Accuracy (ROUGE)**: *57.04%*

**Similarity (ROUGE)**: *70.59%*

**Score (ROUGE)**: *63.10%*

In [19]:
metrics_df = pd.DataFrame(
    {
        "Model": [
            "Perfect Meaning",
            "Perfect Simplification",
            "GPT-4",
            "GPT-4 Zero-Shot",
            "GPT-4o Zero-Shot",
            "Mistral-7B Zero-Shot",
            "Mistral-7B Fine-Tuned",
            "GPT Zero-Shot",
            "Davinci Fine-Tuned",
            "GPT Fine-Tuned",
        ],
        "Accuracy": [
            perfect_meaning_metric_bert.accuracy,
            perfect_simplification_metric_bert.accuracy,
            gpt4_metric_bert.accuracy,
            gpt4_zero_shot_metric_bert.accuracy,
            gpt4_o_metric_bert.accuracy,
            mistral_zero_shot_metric_bert.accuracy,
            mistral_fine_tuned_metric_bert.accuracy,
            gpt_zero_shot_metric_bert.accuracy,
            davinci_finetuned_metric_bert.accuracy,
            gpt_finetuned_metric_bert.accuracy,
        ],
        "Similarity (BERT)": [
            perfect_meaning_metric_bert.similarity,
            perfect_simplification_metric_bert.similarity,
            gpt4_metric_bert.similarity,
            gpt4_zero_shot_metric_bert.similarity,
            gpt4_o_metric_bert.similarity,
            mistral_zero_shot_metric_bert.similarity,
            mistral_fine_tuned_metric_bert.similarity,
            gpt_zero_shot_metric_bert.similarity,
            davinci_finetuned_metric_bert.similarity,
            gpt_finetuned_metric_bert.similarity,
        ],
        "Similarity (BLEU)": [
            perfect_meaning_metric_bleu.similarity,
            perfect_simplification_metric_bleu.similarity,
            gpt4_metric_bleu.similarity,
            gpt4_zero_shot_metric_bleu.similarity,
            gpt4_o_metric_bleu.similarity,
            mistral_zero_shot_metric_bleu.similarity,
            mistral_fine_tuned_metric_bleu.similarity,
            gpt_zero_shot_metric_bleu.similarity,
            davinci_finetuned_metric_bleu.similarity,
            gpt_finetuned_metric_bleu.similarity,
        ],
        "Similarity (ROUGE)": [
            perfect_meaning_metric_rouge.similarity,
            perfect_simplification_metric_rouge.similarity,
            gpt4_metric_rouge.similarity,
            gpt4_zero_shot_metric_rouge.similarity,
            gpt4_o_metric_rouge.similarity,
            mistral_zero_shot_metric_rouge.similarity,
            mistral_fine_tuned_metric_rouge.similarity,
            gpu_zero_shot_metric_rouge.similarity,
            daivnci_finetuned_metric_rouge.similarity,
            gpt_finetuned_metric_rouge.similarity,
        ],
        "Score (BERT)": [
            perfect_meaning_metric_bert.score,
            perfect_simplification_metric_bert.score,
            gpt4_metric_bert.score,
            gpt4_zero_shot_metric_bert.score,
            gpt4_o_metric_bert.score,
            mistral_zero_shot_metric_bert.score,
            mistral_fine_tuned_metric_bert.score,
            gpt_zero_shot_metric_bert.score,
            davinci_finetuned_metric_bert.score,
            gpt_finetuned_metric_bert.score,
        ],
        "Score (BLEU)": [
            perfect_meaning_metric_bleu.score,
            perfect_simplification_metric_bleu.score,
            gpt4_metric_bleu.score,
            gpt4_zero_shot_metric_bleu.score,
            gpt4_o_metric_bleu.score,
            mistral_zero_shot_metric_bleu.score,
            mistral_fine_tuned_metric_bleu.score,
            gpt_zero_shot_metric_bleu.score,
            davinci_finetuned_metric_bleu.score,
            gpt_finetuned_metric_bleu.score,
        ],
        "Score (ROUGE)": [
            perfect_meaning_metric_rouge.score,
            perfect_simplification_metric_rouge.score,
            gpt4_metric_rouge.score,
            gpt4_zero_shot_metric_rouge.score,
            gpt4_o_metric_rouge.score,
            mistral_zero_shot_metric_rouge.score,
            mistral_fine_tuned_metric_rouge.score,
            gpu_zero_shot_metric_rouge.score,
            daivnci_finetuned_metric_rouge.score,
            gpt_finetuned_metric_rouge.score,
        ],
    }
)

metrics = metrics_df.sort_values("Score (BERT)", ascending=False)

# Display metrics
display(Markdown("### Metrics"))
display(metrics)

# Save metrics
path = os.path.join(pwd, "results", "text_simplification", "Metrics")
if not os.path.exists(path):
    os.makedirs(path)
metrics.to_csv(os.path.join(path, "metrics.csv"), index=False)

### Metrics

,Model,Accuracy,Similarity (BERT),Similarity (BLEU),Similarity (ROUGE),Score (BERT),Score (BLEU),Score (ROUGE)
2,GPT-4,0.634931,0.885688,0.012408,0.388889,0.739634,0.024341,0.482346
6,Mistral-7B Fine-Tuned,0.593979,0.913803,0.074866,0.870370,0.719971,0.132972,0.706091
9,GPT Fine-Tuned,0.570417,0.914014,0.041305,0.705882,0.702450,0.077032,0.630961
7,GPT Zero-Shot,0.526722,0.933759,0.134486,1.000000,0.673520,0.214264,0.690004
3,GPT-4 Zero-Shot,0.510284,0.934294,0.094240,0.875000,0.660062,0.159097,0.644631
5,Mistral-7B Zero-Shot,0.473959,0.931159,0.294438,0.884615,0.628177,0.363228,0.617222
4,GPT-4o Zero-Shot,0.455736,0.890117,0.095618,0.000000,0.602827,0.158072,0.000000
8,Davinci Fine-Tuned,0.435822,0.827054,0.007003,0.526316,0.570837,0.013785,0.476813
1,Perfect Simplification,0.361290,0.809957,0.000082,0.000000,0.499689,0.000165,0.000000
0,Perfect Meaning,0.315404,1.000000,0.980000,1.000000,0.479555,0.477220,0.479555


In [21]:
# Keep one sentence of every dataset
example_df = pd.concat(
    [
        metric_test_df.sample(1),
        mistral_zero_shot_df.sample(1),
        mistral_fine_tuned_df.sample(1),
        gpt4_zero_shot_df.sample(1),
        gpt4_o_df.sample(1),
        gpt_fn_df.sample(1),
        gpt_df.sample(1),
        davinci_fn_df.sample(1),
        test_df_3.sample(1),
        test_df_4.sample(1),
    ],
)
example_df.index = (
    ["GPT-4"] * 1
    + ["Mistral-7B Zero-Shot"] * 1
    + ["Mistral-7B Fine-Tuned"] * 1
    + ["GPT-4 Zero-Shot"] * 1
    + ["GPT-4o Zero-Shot"] * 1
    + ["GPT Fine-Tuned"] * 1
    + ["GPT Zero-Shot"] * 1
    + ["Davinci Fine-Tuned"] * 1
    + ["Perfect Meaning"] * 1
    + ["Perfect Simplification"] * 1
)
# Compute Accuracy and Similarity
## Accuracy
accuracy = pd.Series(
    get_simplification_accuracy(
        example_df, "french_difficulty", os.path.join(pwd, "scratch"), m=False
    ).accuracy
)
accuracy.index = example_df.index
example_df["Accuracy"] = accuracy
## Similarity
similarity = get_simplification_similarity(example_df).similarity
example_df["Similarity"] = similarity
## Score
score = (
    4
    * (w_1_bert * accuracy * (1 - w_1_bert) * similarity)
    / (w_1_bert * accuracy + (1 - w_1_bert) * similarity)
)
example_df["Score (BERT)"] = score

# Save
path = os.path.join(pwd, "results", "text_simplification", "Metrics")
if not os.path.exists(path):
    os.makedirs(path)
example_df.to_csv(os.path.join(path, "example.csv"))
example_df.sort_values("Score (BERT)", ascending=False)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: camembert-base
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,Original,Simplified,Accuracy,Similarity,Score (BERT)
GPT-4 Zero-Shot,"Un procès, par ailleurs coûteux et technique, ...","Pour l'instant, il n'est pas prévu de faire un...",0.843566,0.929793,0.884583
GPT Zero-Shot,Je ne vois aucune contradiction entre la promo...,Je pense que promouvoir les langues régionales...,0.840599,0.898634,0.868649
GPT Fine-Tuned,"Aussi, depuis vingt ans, s'eston beaucoup occu...","Également, ces vingt dernières années, on a be...",0.779209,0.920690,0.844062
Davinci Fine-Tuned,"Un frisson me saisit soudain, non pas un friss...","On dira que j'ai eu un frisson, mais pas un fr...",0.646375,0.776540,0.705504
Mistral-7B Fine-Tuned,"Comme la conscience religieuse, la conscience ...",La conscience morale est aussi importante que ...,0.590418,0.861848,0.700768
Mistral-7B Zero-Shot,Si de la pluie tombe alors qu'il y a du soleil...,"Si il pleut et qu'il y a du soleil, vous pourr...",0.459774,0.950336,0.619725
GPT-4o Zero-Shot,Il lui semblait que le froid était arrivé d'un...,Il avait l'impression que le froid était arriv...,0.432812,0.986223,0.601605
GPT-4,Le réalisateur utilise la lumière et l'ombre p...,Le réalisateur fait des effets avec la lumière...,0.400798,0.892963,0.553267
Perfect Meaning,C'est en Italie que les deux mots sont le plus...,C'est en Italie que les deux mots sont le plus...,0.306157,1.000000,0.468791
Perfect Simplification,Or les éleveurs du Montana craignent que les b...,"Quel enfantillage, d'ailleurs, de croire à la ...",0.143286,0.804113,0.243231
